## Задание

**Вам необходимо решить задачу любыми способами, которые вы считаете верными.**

Задача: имеется предметная область, которая имеет иерархическую структуру показателей. Например, Город -> Улица -> Дом -> Подъезд -> Этаж -> Квартира -> Показание счетчика. Вам необходимо сгенерировать предметную область самостоятельно (которая вам нравится), представить ее в выбранной вами структуре данных. Написать функцию, которая ищет все доступные комбинации показателей. Функция должна быть реализована в переиспользуемом виде на случай, если предметная область изменится.

Ниже приложите все свои функции/классы, которыми вы пользовались для решения задачи. Также можете отправить дополнительно файлы, которые вы считаете нужны добавить. Продемонстрируйте работу ваших функций/классов.

---

ТЗ: 

1. Город
2. Город - Улица
3. Город - Улица - Дом
Словарь, поле city = 'moscow', street = 'uralskay',

Как нулевой элемент списка - входной словарь

Возвращаем словарь с восстановленным путем

И словари со всеми показателями следующего уровня

Слорваь вида {city: street: 'название', 

В каждом словаре

Список словарей (не список названий!!)

Словари все должны быть полными

Там где фильтрации нет - Nan.

Следующий этап - восстановление улицы.

---

1. Важен ли порядок во входном словаре?
2. Важны ли названия ключей?
3. None как элемент словаря или может быть и отдельно?
4. А если несколько None?

In [15]:
# пример структуры
hierarchy = {
    'moscow': {
        'None':
            {'1':
                {
                    'None': {
                        'вода': {}, 'электричество': {}
                    }  
                }, 
            '2':
                {
                'None': {
                            'вода': {}, 'электричество': {}
                        },
                '2': {
                            'вода': {}, 'электричество': {}
                        },
                '36': {
                            'вода': {}, 'электричество': {}
                        },
                '76': {
                            'вода': {}, 'электричество': {}
                        },
                }
        },
        'arbat':
            {'None':
                {
                    'None': 
                        {
                            'вода': {}, 'электричество': {}
                        }    
                }, 
            '2':
                {
                    'None': 
                    {
                        'вода': {}, 'электричество': {}
                    } 
                },
                
            '3':
                {
                    '4': 
                    {
                        'вода': {}, 'электричество': {}
                    }     
                },
            '7':
                {
                    'None': 
                    {
                        'вода': {}, 'электричество': {}
                    }
                }
        },
        'novoslobodskaya':
        {'2':
            {
               '3': {
                        'вода': {}, 'электричество': {}
                    } 
            },      
        '3':
            {
                'None': {
                    'вода': {}, 'электричество': {}
                }     
            }
        }
    },
    'spb': {
        'uralskaya': 
            {'2':
                {
                'None': {
                            'вода': {}, 'электричество': {}
                        } 
                },
                
            '3':
                {
                    'None': {
                        'вода': {}, 'электричество': {}
                    }     
                }
        },
        'rubinshteina':
        {'2':
            {
                'None': {
                        'вода': {}, 'электричество': {}
                    } 
            },
            
            '3':
                {
                    'None': {
                        'вода': {}, 'электричество': {}
                    }     
                }
        }
    }
}

In [ ]:
def get_query_list(hierarchy, query):
    """ Функция, возвращающая список словарей всевозможных комбинаций 
    из соответсвующих значений переданного объекта

    Аргументы:
        hierarchy (dict): структура, по которой ведется поиск
        query (dict): запрос
    """
    # список отсортированых по ключам значений
    values = [query[item] for item in sorted(query.keys())]
    
    if values[-1] == 'None':
        raise ValueError('driver не может быть None!')
    # ответ
    answer = []
    
    # функция, возвращающая последний не None списка, уровень рекурсии 
    # и флажок, показывающий наличие None в запросе
    def iter_levels(hierarchy, values, last_not_nan = None, level_num = 0, has_none = False):
        # критерий остановки рекурсии
        if level_num == len(values)-1 or not isinstance(hierarchy, dict) or len(hierarchy) == 0:
            return hierarchy, last_not_nan, level_num, has_none
        
        val = values[level_num]
        if val not in hierarchy:
            # ЗДЕСЬ ДОЛЖНО БЫТЬ ВОССТАНОВЛЕНИЕ ПУТИ ПРИ ПРОПУСКЕ УРОВНЯ 
            raise ValueError(f'Нет значения {val} на уровне {level_num} в json!')
        
        if val != 'None':
            last_not_nan = val
        else:
            has_none = True
        
        return iter_levels(hierarchy[val], values, last_not_nan, level_num+1, has_none)
    
    # находим последнее не None и уровень, на котором остановилась рекурсия
    new_hierarchy, last_not_nan, level_num, has_none = iter_levels(hierarchy, values)
    
    # если не блоы None - просто возвращаем запрошенное значение
    if not has_none:
        return new_hierarchy[values[-1]]
    
    
    # создаем копию переданной структуры
    new_hierarchy = hierarchy.copy()
    # сохраняем значения двух первых уровней
    level_not_nan = 1
    curr_val = values[level_not_nan]
    prev_val = values[level_not_nan-1]
    
    # циклом доходим до нужной иерархии (сверху последнего не None)
    while curr_val != last_not_nan:
        new_hierarchy = new_hierarchy[prev_val]
        level_not_nan = level_not_nan + 1
        
        prev_val = curr_val
        curr_val = values[level_not_nan]
        
    # создаем список значений данного уровня
    if isinstance(new_hierarchy, dict):
        item_list = new_hierarchy.keys()
    else:
        item_list = new_hierarchy
    # проходимся по значениям и добавляем их в ответ 
    for item in item_list:
        # дублируем словарь, чтобы не изменять исходный
        query_copy = query.copy()
        query_copy[f'lvl_{level_not_nan-1}'] = item
        answer.append(query_copy)
        
    return answer

In [35]:
query_1 = {'lvl_0': 'moscow', 'lvl_1': 'None', 'lvl_2': '2', 'lvl_3': 'None', 'lvl_4': 'вода'}
query_2 = {'lvl_0': 'moscow', 'lvl_1': 'None', 'lvl_2': 'None', 'lvl_3': 'None', 'lvl_4': 'вода'}
query_3 = {'lvl_0': 'moscow', 'lvl_1': 'arbat', 'lvl_2': '3', 'lvl_3': '4', 'lvl_4': 'вода'}

In [37]:
get_query_list(hierarchy, query_1)

[{'lvl_0': 'moscow',
  'lvl_1': 'None',
  'lvl_2': '2',
  'lvl_3': 'None',
  'lvl_4': 'вода'},
 {'lvl_0': 'moscow',
  'lvl_1': 'arbat',
  'lvl_2': '2',
  'lvl_3': 'None',
  'lvl_4': 'вода'},
 {'lvl_0': 'moscow',
  'lvl_1': 'novoslobodskaya',
  'lvl_2': '2',
  'lvl_3': 'None',
  'lvl_4': 'вода'}]

----

*Эталонный ответ:*

In [ ]:
answer = [{'lvl_0': 'moscow', 'lvl_1': 'None', 'lvl_2': 'None', 'lvl_3': 'None', 'driver_1': 'вода'}, 
          {'lvl_0': 'moscow', 'lvl_1': 'Arbat', 'lvl_2': 'None', 'lvl_3': 'None', 'driver_1': 'вода'},
          {'lvl_0': 'moscow', 'lvl_1': 'Kyzne4niy', 'lvl_2': 'None', 'lvl_3': 'None', 'driver_1': 'вода'},
          {'lvl_0': 'moscow', 'lvl_1': 'Lenina', 'lvl_2': 'None', 'lvl_3': 'None', 'driver_1': 'вода'},
          {'lvl_0': 'moscow', 'lvl_1': 'Pushkina', 'lvl_2': 'None', 'lvl_3': 'None', 'driver_1': 'вода'},
          {'lvl_0': 'moscow', 'lvl_1': 'Arbat', 'lvl_2': '1', 'lvl_3': 'None', 'driver_1': 'вода'},
          {'lvl_0': 'moscow', 'lvl_1': 'Arbat', 'lvl_2': '2', 'lvl_3': 'None', 'driver_1': 'вода'},
          {'lvl_0': 'moscow', 'lvl_1': 'Arbat', 'lvl_2': '3', 'lvl_3': 'None', 'driver_1': 'вода'}
          ]